# MLG HW3

In [1]:
import pandas as pd
import numpy as np
import os, glob, json
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import recall_score
from sklearn.metrics import ndcg_score


import torch
from deepctr_torch.inputs import SparseFeat, get_feature_names
import torchfm



config = {'test_size' : 0.2, 
          'n_epoch' : 20}
rmse = {}
ndcg_10 = {}
recall_10 = {}


with open("./metrics/rmse.txt", "r") as fp:
    rmse = json.load(fp)
with open("./metrics/ndcg.txt", "r") as fp:
    ndcg_10 = json.load(fp)
# with open("./metrics/recall.txt", "r") as fp:
#     recall_10 = json.load(fp)
print(rmse)
print(ndcg_10)

temp = pd.read_csv('results.csv', header = [0, 1], index_col = [0])

/home/rita/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-19 15:17:26.913863: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 15:17:28.450314: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-19 15:17:36.076780: W tensorflow/compiler/xla/stream_executor/pla

{'bookIPNN': 0.7630446736162781, 'bookOPNN': 0.7596840523834419, 'bookPNN': 0.7518341524053948, 'bookCCPM': 0.7424491460611815, 'bookWDL': 0.7093089364684898, 'bookDCN': 0.7302810220618285, 'bookNFM': 0.7568314381827344, 'bookDeepFM': 0.7386060319249964, 'bookAFM': 0.7353113174115252, 'bookxDeepFM': 0.7524432051769194, 'movieIPNN': 1.1257715623148956, 'movieOPNN': 1.0723128454116133, 'moviePNN': 1.1457441626815474, 'movieCCPM': 1.053634406962656, 'movieWDL': 1.0492728323684728, 'movieDCN': 1.0530302980259754, 'movieNFM': 1.0942265873231904, 'movieDeepFM': 1.0515179004479969, 'movieAFM': 1.0710224032118236, 'moviexDeepFM': 1.0744275480292602, 'businessIPNN': 1.2039563606524462, 'businessOPNN': 1.1574377340755466, 'businessPNN': 1.1446729801067166, 'businessCCPM': 1.0781101968351026, 'businessWDL': 1.0570848609478811, 'businessDCN': 1.0804563918023302, 'businessNFM': 1.293179055678124, 'businessDeepFM': 1.0519467034898193, 'businessAFM': 1.083461280218211, 'businessxDeepFM': 1.1471864908

## Read Data

In [2]:
def Data2List(path):
    data = {}
    os.chdir(path)
    files = glob.glob('*.dat')
    file_name = [x[:-4] for x in files]
    for i in range(len(file_name)):
        data[file_name[i]] = pd.read_csv(files[i], sep = '\t', header = None)
        # print(file_name[i])
        # print(data[file_name[i]].head(2))
    return data

# Data Filtering
# delete users who's interaction less than 3
def del_less_than_3(data, target = 'user') :
    temp = data[target].value_counts()
    temp = temp.index[temp.values > 3]
    data = data[data[target].isin(temp)]
    
    return data

DoubanBook = Data2List('/home/rita/111/111-2MLG/HW3/data/DoubanBook')
Movielens = Data2List('/home/rita/111/111-2MLG/HW3/data/Movielens')
Yelp = Data2List('/home/rita/111/111-2MLG/HW3/data/Yelp')

DoubanBook['user_book'].columns = ['user', 'book', 'rating']
Movielens['user_movie'].columns = ["user","movie","rating","time"]
Yelp['user_business'].columns = ['user', 'business', 'rating']

DoubanBook['user_book'] = del_less_than_3(DoubanBook['user_book'])
Movielens['user_movie'] = del_less_than_3(Movielens['user_movie'])
Yelp['user_business'] = del_less_than_3(Yelp['user_business'])

# Merge user's feature
# DoubanBook['user'] = pd.merge(left = DoubanBook['user_location'], right = DoubanBook['user_group'], on = 0, how = 'outer')
# DoubanBook['user'].columns = ['user', 'location', 'group']
# DoubanBook['user'] = pd.merge(left = DoubanBook['user_book'], right = DoubanBook['user'], on = 'user', how = 'left')
DoubanBook['user'] = pd.merge(left = DoubanBook['user_book'], right = DoubanBook['user_location'], left_on = 'user', right_on = 0, how = 'left')
DoubanBook['user'] = DoubanBook['user'].drop([0], axis = 1)
DoubanBook['user'].columns = ['user', 'book', 'rating', 'location']

Movielens['user'] = pd.merge(left = Movielens['user_age'], right = Movielens['user_occupation'], on = 0, how = 'outer')
Movielens['user'].columns = ['user', 'age', 'occupation']
Movielens['user'] = pd.merge(left = Movielens['user_movie'], right = Movielens['user'], on = 'user', how = 'left')

Yelp['user'] = Yelp['user_business']   

print(DoubanBook['user'].shape)
print(DoubanBook['user'].head(2))
print(Movielens['user'].shape)
print(Movielens['user'].head(2))
print(Yelp['user'].shape)
print(Yelp['user'].head(2))
path = '/home/rita/111/111-2MLG/HW3'
os.chdir(path)

(788898, 4)
    user  book  rating  location
0  10855   938       4      33.0
1  10027     3       3     394.0
(100000, 6)
   user  movie  rating       time  age  occupation
0   196    242       3  881250949    5           3
1   186    302       3  891717742    4           4
(184835, 3)
   user  business  rating
2     2       186       5
3     2       205       5


In [3]:
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4])
kf = KFold(n_splits=3)
kf.get_n_splits(X)
print(kf)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


KFold(n_splits=3, random_state=None, shuffle=False)
TRAIN: [2 3] TEST: [0 1]
TRAIN: [0 1 3] TEST: [2]
TRAIN: [0 1 2] TEST: [3]


## 10 Typical RecSys Methods
**UCF-s UCF-p ICF-s ICF-p** **MF** **FM** BPR-MF BPR-FM GBDT+LR XGB+LR 

In [100]:
# CF = Collaborative Filtering
# UCF-s / UCF-p / ICF-s / ICF-p
# https://elevenzou.github.io/2019/02/16/Surprise%E5%BA%93%E5%9F%BA%E7%A1%80%E7%94%A8%E6%B3%95/
# 4 mins
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import KNNBasic
from surprise.accuracy import rmse
def Based_CF(data, feats, based = 'U', sim = 's') :
    print('Start Training {}CF-{} model'.format(based, sim))
    reader = Reader(rating_scale=(1,5))
    data = Dataset.load_from_df(data[feats], reader)
    based_dict = {'U' : True, 'I' : False}
    sim_dict = {'s' : "cosine", 'p' : 'pearson'}
    sim_options = {"name":sim_dict[sim],"user-based":based_dict[based]}
    algo = KNNBasic(sim_options=sim_options)
    a = cross_validate(algo, data, cv=5, verbose=True, n_jobs = 5)
    print()
    return a['test_rmse'].mean()

with open("./metrics/rmse.txt", "r") as fp:
    rmse = json.load(fp)
with open("./metrics/ndcg.txt", "r") as fp:
    ndcg_10 = json.load(fp)
with open("./metrics/recall.txt", "r") as fp:
    recall_10 = json.load(fp)

data_ls = [DoubanBook['user_book'], Movielens['user_movie'], Yelp['user_business']]
feat_ls = [['user', 'book', 'rating'], ["user", "movie", "rating"], ['user', 'business', 'rating']]
based = ['U', 'I']
sim = ['s', 'p']
for data, feats in zip(data_ls, feat_ls) :
    # print(feats)
    for b in based :
        for s in sim : 
            name = feats[1] + '{}CF-{}'.format(b, s)
            rmse_t = Based_CF(data, feats, b, s)
            rmse[name] = rmse_t

with open("./metrics/rmse.txt", "w") as fp:
    json.dump(rmse, fp)
with open("./metrics/ndcg.txt", "w") as fp:
    json.dump(ndcg_10, fp)
with open("./metrics/recall.txt", "w") as fp:
    json.dump(recall_10, fp)

Start Training UCF-s model
Computing the cosine similarity matrix...
Computing the cosine similarity matrix...
Computing the cosine similarity matrix...
Computing the cosine similarity matrix...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7529  0.7576  0.7573  0.7601  0.7544  0.7565  0.0025  
MAE (testset)     0.6053  0.6089  0.6085  0.6105  0.6061  0.6079  0.0019  
Fit time          14.10   13.05   13.31   12.92   13.18   13.31   0.41    
Test time         19.75   19.49   19.81   19.65   19.75   19.69   0.11    

Start Training UCF-p model
Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Co

In [10]:
# MF
# https://albertauyeung.github.io/2017/04/23/python-matrix-factorization.html/
# 9 mins
class Matrix_Factorization():

    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.

        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """

        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            # if (i+1) % 10 == 0:
            if (i+1) % 1 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

def MF(data, feats, target, K = 2, alpha = 0.01, beta = 0.01) :
    print(feats)
    train, test = train_test_split(data, test_size = config['test_size'])
    a = data[feats[0]].max()
    b = data[feats[1]].max()
    R = np.zeros((a, b), dtype = np.float128)
    train_x = np.array(train[feats])
    train_y = np.array(train[target])
    test_x = np.array(test[feats])
    test_y = np.array(test[target])
    for i in range(train_x.shape[0]):
        # R[train.iat[i, 0] - 1, train.iat[i, 1] - 1] = train.iat[i, 2]
        # R[train.at[i, feats[0]] - 1, train.at[i, feats[1]] - 1] = train.at[i, target]
        R[train_x[i, 0] - 1, train_x[i, 1] - 1] = train_y[i]
    

    mf = Matrix_Factorization(R, K=K, alpha=alpha, beta=beta, iterations=config['n_epoch'])
    training_process = mf.train()

    temp = mf.full_matrix()

    t = []
    for i in range(test.shape[0]) :
        t.append(temp[test_x[i, 0] - 1, test_x[i, 1] - 1])
    t = np.array(t)
    
    test_y = np.where(test_y > 3, 1, 0)
    t = np.where(t > 3, 1, 0)
    
    rmse = sqrt(mean_squared_error(test_y, t))
    ndcg = ndcg_score(test_y.reshape(1, -1), t.reshape(1, -1), k=10)
    recall = recall_score(test_y.reshape(1, -1).squeeze(), t.reshape(1, -1).squeeze())
    
    print("test MSE : {:.4f}".format(rmse, 4), '\n')    
    
    return  rmse, ndcg, recall
    # print("Global bias:", mf.b, "User bias:", mf.b_u, "Item bias:", mf.b_i, sep = '\n')
    # x = [x for x, y in training_process]
    # y = [y for x, y in training_process]
    # plt.figure(figsize=((16,4)))
    # plt.plot(x, y)
    # plt.xticks(x, x)
    # plt.xlabel("Iterations")
    # plt.ylabel("Mean Square Error")
    # plt.grid(axis="y")
with open("./metrics/rmse.txt", "r") as fp:
    rmse = json.load(fp)
with open("./metrics/ndcg.txt", "r") as fp:
    ndcg_10 = json.load(fp)
with open("./metrics/recall.txt", "r") as fp:
    recall_10 = json.load(fp)
   
data_ls = [DoubanBook['user_book'], Movielens['user_movie'], Yelp['user_business']]
feats_ls = [['user', 'book', 'rating'], ["user", "movie", "rating"], ['user', 'business', 'rating']]
for data, feats in zip(data_ls, feats_ls):
    name = feats[1] + 'MF'
    rmse_t, ndcg_t, recall_t = MF(data, feats[:-1], feats[-1], K = 5, alpha = 0.1, beta = 0.15)
    rmse[name] = rmse_t
    ndcg_10[name] = ndcg_t
    recall_10[name] = recall_t
    
with open("./metrics/rmse.txt", "w") as fp:
    json.dump(rmse, fp)
with open("./metrics/ndcg.txt", "w") as fp:
    json.dump(ndcg_10, fp)
with open("./metrics/recall.txt", "w") as fp:
    json.dump(recall_10, fp)


['user', 'book']
Iteration: 1 ; error = 549.2433
Iteration: 2 ; error = 544.3827
Iteration: 3 ; error = 542.5937
Iteration: 4 ; error = 541.1101
Iteration: 5 ; error = 539.6039
Iteration: 6 ; error = 537.5114
Iteration: 7 ; error = 535.5107
Iteration: 8 ; error = 532.3612
Iteration: 9 ; error = 530.4623
Iteration: 10 ; error = 529.1933
Iteration: 11 ; error = 526.3051
Iteration: 12 ; error = 524.2198
Iteration: 13 ; error = 523.2564
Iteration: 14 ; error = 520.8289
Iteration: 15 ; error = 519.9230
Iteration: 16 ; error = 517.9427
Iteration: 17 ; error = 517.1025
Iteration: 18 ; error = 515.6284
Iteration: 19 ; error = 514.4382
Iteration: 20 ; error = 514.1479
test MSE : 0.4860 

['user', 'movie']
Iteration: 1 ; error = 268.2499
Iteration: 2 ; error = 263.5227
Iteration: 3 ; error = 258.9008
Iteration: 4 ; error = 254.3870
Iteration: 5 ; error = 250.9033
Iteration: 6 ; error = 249.9556
Iteration: 7 ; error = 247.7571
Iteration: 8 ; error = 247.5062
Iteration: 9 ; error = 244.6710
Iterat

In [11]:
# FM
# https://github.com/coreylynch/pyFM
# 3 mins
from pyfm import pylibfm
from sklearn.feature_extraction import DictVectorizer
def FM(data, feats, target) :
    print(feats)
    data = data.astype(str)
    train, test = train_test_split(data, test_size = config['test_size'])
    train_y = pd.Series(train[target].astype(float)).tolist()
    train_x = train[feats].to_dict('records')
    test_y = np.array(test[target].astype(float)) # .tolist()
    test_x = test[feats].to_dict('records')

    v = DictVectorizer()
    X = v.fit_transform(train_x).astype('double')
    fm = pylibfm.FM(num_factors=10, num_iter=config['n_epoch'], verbose=True, task="regression", initial_learning_rate=0.001, learning_rate_schedule="optimal")
    fm.fit(X,train_y)
    temp = fm.predict(v.transform(test_x))

    rmse = sqrt(mean_squared_error(test_y, temp))
    test_y = np.where(test_y > 3, 1, 0)
    temp = np.where(temp > 3, 1, 0)
    ndcg = ndcg_score(test_y.reshape(1, -1), temp.reshape(1, -1), k=10)
    recall = recall_score(test_y.squeeze(), temp.squeeze())
    print("test MSE : {:.4f}".format(rmse, 4), '\n')
    
    return rmse, ndcg, recall

with open("./metrics/rmse.txt", "r") as fp:
    rmse = json.load(fp)
with open("./metrics/ndcg.txt", "r") as fp:
    ndcg_10 = json.load(fp)
with open("./metrics/recall.txt", "r") as fp:
    recall_10 = json.load(fp)
    
data_ls = [DoubanBook['user_book'], Movielens['user_movie'], Yelp['user_business']]
feats_ls = [['user', 'book', 'rating'], ["user", "movie", "rating"], ['user', 'business', 'rating']]
for data, feats in zip(data_ls, feats_ls):
    name = feats[1] + 'FM'
    rmse_t, ndcg_t, recall_t = FM(data, feats[:-1], feats[-1])
    rmse[name] = rmse_t
    ndcg_10[name] = ndcg_t
    recall_10[name] = recall_t
    
with open("./metrics/rmse.txt", "w") as fp:
    json.dump(rmse, fp)
with open("./metrics/ndcg.txt", "w") as fp:
    json.dump(ndcg_10, fp)
with open("./metrics/recall.txt", "w") as fp:
    json.dump(recall_10, fp)

['user', 'book']
Creating validation dataset of 0.01 of training for adaptive regularization
-- Epoch 1
Training MSE: 0.31985
-- Epoch 2
Training MSE: 0.29114
-- Epoch 3
Training MSE: 0.27828
-- Epoch 4
Training MSE: 0.26993
-- Epoch 5
Training MSE: 0.26389
-- Epoch 6
Training MSE: 0.25919
-- Epoch 7
Training MSE: 0.25542
-- Epoch 8
Training MSE: 0.25227
-- Epoch 9
Training MSE: 0.24957
-- Epoch 10
Training MSE: 0.24721
-- Epoch 11
Training MSE: 0.24513
-- Epoch 12
Training MSE: 0.24328
-- Epoch 13
Training MSE: 0.24159
-- Epoch 14
Training MSE: 0.24006
-- Epoch 15
Training MSE: 0.23865
-- Epoch 16
Training MSE: 0.23735
-- Epoch 17
Training MSE: 0.23612
-- Epoch 18
Training MSE: 0.23498
-- Epoch 19
Training MSE: 0.23390
-- Epoch 20
Training MSE: 0.23291
test MSE : 0.7063 

['user', 'movie']
Creating validation dataset of 0.01 of training for adaptive regularization
-- Epoch 1
Training MSE: 0.60339
-- Epoch 2
Training MSE: 0.52638
-- Epoch 3
Training MSE: 0.49771
-- Epoch 4
Training MSE

In [45]:
#not yet
# BPR-FM
# https://github.com/etlundquist/rankfm/blob/master/examples/movielens.ipynb
import os, sys, git
import numba as nb
import warnings

sns.set_style('whitegrid')
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
# plt.rcParams
# git_repo = git.Repo('.', search_parent_directories=True)
# git_root = git_repo.git.rev_parse('--show-toplevel')
# cython_path = os.path.join(git_root, 'rankfm')

# sys.path[0] = git_root
# sys.path[1] = cython_path
# sys.path[:2]
# !cd $git_root && python setup.py build_ext --inplace

# %load_ext autoreload
# %autoreload 2

import rankfm
from rankfm.rankfm import RankFM
from rankfm.evaluation import hit_rate, reciprocal_rank, discounted_cumulative_gain, precision, recall, diversity


# repo_root = "/Users/ericlundquist/Repos/rankfm"
# data_path = os.path.join(repo_root, "data/examples")
# print("\n".join([repo_root, data_path]))

interactions = Movielens['user_movie']
interactions_train, interactions_test = train_test_split(interactions, test_size = config['test_size'])

user_features = Movielens['user']
item_features = Movielens['movie_genre']

item_names = pd.read_csv(os.path.join(data_path, 'ML_1M_ITEM_NAMES.csv'))



unique_users = interactions.user_id.nunique()
unique_items = interactions.item_id.nunique()

print("interactions shape: {}".format(interactions.shape))
print("interactions unique users: {}".format(interactions.user_id.nunique()))
print("interactions unique items: {}".format(interactions.item_id.nunique()))

print("user features users:", interactions.user_id.nunique())
print("item features items:", interactions.item_id.nunique())



sparsity = 1 - (len(interactions) / (unique_users * unique_items))
print("interaction matrix sparsity: {}%".format(round(100 * sparsity, 1)))


np.random.seed(1492)
interactions['random'] = np.random.random(size=len(interactions))
test_pct = 0.25


train_mask = interactions['random'] <  (1 - test_pct)
valid_mask = interactions['random'] >= (1 - test_pct)

interactions_train = interactions[train_mask][['user_id', 'item_id']]
interactions_valid = interactions[valid_mask][['user_id', 'item_id']]

train_users = np.sort(interactions_train.user_id.unique())
valid_users = np.sort(interactions_valid.user_id.unique())
cold_start_users = set(valid_users) - set(train_users)

train_items = np.sort(interactions_train.item_id.unique())
valid_items = np.sort(interactions_valid.item_id.unique())
cold_start_items = set(valid_items) - set(train_items)

print("train shape: {}".format(interactions_train.shape))
print("valid shape: {}".format(interactions_valid.shape))

print("train users: {}".format(len(train_users)))
print("valid users: {}".format(len(valid_users)))
print("cold-start users: {}".format(cold_start_users))

print("train items: {}".format(len(train_items)))
print("valid items: {}".format(len(valid_items)))
print("cold-start items: {}".format(cold_start_items))

user_features = user_features[user_features.user_id.isin(train_users)]
item_features = item_features[item_features.item_id.isin(train_items)]
user_features.shape, item_features.shape

model = RankFM(factors=20, loss='warp', max_samples=20, alpha=0.01, sigma=0.1, learning_rate=0.10, learning_schedule='invscaling')
model


%%time
model.fit(interactions_train, epochs=20, verbose=True)

valid_scores = model.predict(interactions_valid, cold_start='nan') 
print(valid_scores.shape)
pd.Series(valid_scores).describe()

valid_recommendations = model.recommend(valid_users, n_items=10, filter_previous=True, cold_start='nan')
valid_recommendations.head()

k = 10

most_popular = interactions_train.groupby('item_id')['user_id'].count().sort_values(ascending=False)[:k]
most_popular

test_user_items = interactions_valid.groupby('user_id')['item_id'].apply(set).to_dict()
test_user_items = {key: val for key, val in test_user_items.items() if key in set(train_users)}

base_hrt = np.mean([int(len(set(most_popular.index) & set(val)) > 0)                       for key, val in test_user_items.items()])
base_pre = np.mean([len(set(most_popular.index) & set(val)) / len(set(most_popular.index)) for key, val in test_user_items.items()])
base_rec = np.mean([len(set(most_popular.index) & set(val)) / len(set(val))                for key, val in test_user_items.items()])

print("number of test users: {}".format(len(test_user_items)))
print("baseline hit rate: {:.3f}".format(base_hrt))
print("baseline precision: {:.3f}".format(base_pre))
print("baseline recall: {:.3f}".format(base_rec))




%%time
model_hit_rate = hit_rate(model, interactions_valid, k=k)
model_reciprocal_rank = reciprocal_rank(model, interactions_valid, k=k)
model_dcg = discounted_cumulative_gain(model, interactions_valid, k=k)
model_precision = precision(model, interactions_valid, k=k)
model_recall = recall(model, interactions_valid, k=k)


print("hit_rate: {:.3f}".format(model_hit_rate))
print("reciprocal_rank: {:.3f}".format(model_reciprocal_rank))
print("dcg: {:.3f}".format(model_dcg, 3))
print("precision: {:.3f}".format(model_precision))
print("recall: {:.3f}".format(model_recall))

recommendation_diversity = diversity(model, interactions_valid, k=k)
recommendation_diversity.head(10)

top_items = pd.merge(item_names, recommendation_diversity, on='item_id', how='inner')
top_items = top_items.set_index('item_id').loc[recommendation_diversity.item_id].reset_index()
top_items = top_items[['item_id', 'cnt_users', 'pct_users', 'title', 'genres']]
top_items.head(10)

coverage = np.mean(recommendation_diversity['cnt_users'] > 0)
print("percentage of items recommended to at least one user: {:.3f}".format(coverage))

nonzero_users = recommendation_diversity[recommendation_diversity.cnt_users > 0]
entropy = -np.sum(nonzero_users['pct_users'] * np.log2(nonzero_users['pct_users']))
print("entropy value of recommended items: {:.3f}".format(entropy))

N = 50
fig, axes = plt.subplots(1, 1, figsize=[16, 4])

topN = recommendation_diversity.iloc[:N, :]
axes.bar(topN.index.values + 1, topN.pct_users, width=1, edgecolor='black', alpha=0.75)
axes.set(xlabel='Item Rank', ylabel='Percentage of Users', title='Percentage of Users Recommended by Item Rank')
plt.show()


random_user = np.random.choice(interactions_valid.user_id.unique())
print("random user: {}".format(random_user))

random_user_recs = valid_recommendations.loc[random_user]
random_user_recs = item_names[item_names.item_id.isin(random_user_recs)].set_index('item_id').loc[random_user_recs]
random_user_recs

most_similar_items = model.similar_items(919)
most_similar_items = item_names.set_index('item_id').loc[most_similar_items]
most_similar_items

most_similar_items = model.similar_items(2028)
most_similar_items = item_names.set_index('item_id').loc[most_similar_items]
most_similar_items



ImportError: cannot import name 'RankFM' from 'rankfm.rankfm' (/home/rita/111/111-2MLG/HW3/rankfm/rankfm/__init__.py)

## 10 NN-based RecSys Methods
FNN **IPNN** **OPNN** **PIN** **CCPM** NeuMF **WD** **DeepCross** **NFM** **DeepFM** 

In [12]:
#NN-based model
from deepctr_torch.models import PNN # PIN
from deepctr_torch.models import CCPM # CCPM

from deepctr_torch.models import WDL # WD
from deepctr_torch.models import DCN # DeepCross
from deepctr_torch.models import NFM # NFM
from deepctr_torch.models import DeepFM # DeepFM

#recent nn-based approach
from deepctr_torch.models import AFM # AFM
from deepctr_torch.models import xDeepFM # xDeePFM

# new



In [13]:

class nn_based :
    def __init__(self, data, features, target, device = 'cuda') : 
        self.features = features
        self.target = target
        for feat in self.features :
            enc = LabelEncoder()
            data[feat] = enc.fit_transform(data[feat])
        # SparseFeat(name, vocabulary_size, embedding_dim, use_hash, dtype, embedding_name, group_name)
        # Dense : Numeric 、 Sparse : Category 、 Sequence : Time Series       
        fixed = [SparseFeat(feat, data[feat].nunique()) for feat in self.features]
        linear_feats = fixed
        dnn_feats = fixed
        feature_names = get_feature_names(linear_feats + dnn_feats)

        # train test split
        self.train, self.test = train_test_split(data, test_size = 0.2)
        self.train_data = {name: self.train[name] for name in feature_names}
        self.test_data = {name: self.test[name] for name in feature_names}

        self.model_dict = {"IPNN":PNN(dnn_feats,use_inner=True,use_outter=False,task='regression',device=device),
              "OPNN":PNN(dnn_feats,use_inner=False,use_outter=True,task='regression',device=device),
              "PNN":PNN(dnn_feats,use_inner=True,use_outter=True,task='regression',device=device),
              "CCPM":CCPM(linear_feats, dnn_feats, task='regression',device=device),
             "WDL":WDL(linear_feats, dnn_feats, task='regression',device=device),
             "DCN":DCN(linear_feats, dnn_feats, task='regression',device=device),
             "NFM":NFM(linear_feats, dnn_feats, task='regression',device=device),
             "DeepFM":DeepFM(linear_feats, dnn_feats, task='regression',device=device),
             "AFM":AFM(linear_feats, dnn_feats, task='regression',device=device),
             "xDeepFM":xDeepFM(linear_feats, dnn_feats, task='regression',device=device), 
            #  ""
             }
             
    def train_model(self, model, n_epochs = 10):
        print(self.features, model, '==================', sep = '\n')
        model = self.model_dict[model]

        model.compile('adam', 'mse', metrics = ['mse'])
        history = model.fit(self.train_data,self.train[self.target].values,batch_size=256, epochs=n_epochs, verbose=5, validation_split=0.1)
        pred_ans = model.predict(self.test_data, batch_size=256)
        
        rmse = sqrt(mean_squared_error(self.test[self.target].values, pred_ans))
        
        test_y = np.where(self.test[self.target].values > 3, 1, 0)
        pred_ans = np.where(pred_ans > 3, 1, 0)
        
        ndcg = ndcg_score(test_y.reshape(1, -1), pred_ans.T, k=10)
        recall10 = recall_score(test_y.reshape(1, -1).squeeze(), pred_ans.squeeze())
        
        print("test MSE : {:.4f}".format(rmse, 4))
        print("test ndcg : {:.4f}".format(ndcg, 4))
        # print("test recall10 : {:.4f}".format(recall10, 4))
        
        return rmse, ndcg, recall10


In [14]:
# 73 min
data_ls = [DoubanBook['user'], Movielens['user'], Yelp['user']]
feats_ls = [['user', 'book', 'location'], ['user', 'movie', 'time', 'age', 'occupation'], ['user', 'business']]
model = ['IPNN', 'OPNN', 'PNN', 'CCPM', 'WDL', 'DCN', 'NFM', 'DeepFM', 'AFM', 'xDeepFM']

with open("./metrics/rmse.txt", "r") as fp:
    rmse = json.load(fp)
with open("./metrics/ndcg.txt", "r") as fp:
    ndcg_10 = json.load(fp)
with open("./metrics/recall.txt", "r") as fp:
    recall_10 = json.load(fp)    
    
for data, feats in zip(data_ls, feats_ls) :
    nn = nn_based(data, feats, 'rating')
    for m in model :
        rmse_t, ndcg_t, recall_t = nn.train_model(m, config['n_epoch'])
        name = feats[1] + m
        rmse[name] = rmse_t
        ndcg_10[name] = ndcg_t
        recall_10[name] = recall_t

with open("./metrics/rmse.txt", "w") as fp:
    json.dump(rmse, fp)
with open("./metrics/ndcg.txt", "w") as fp:
    json.dump(ndcg_10, fp)
with open("./metrics/recall.txt", "w") as fp:
    json.dump(recall_10, fp)

['user', 'book', 'location']
IPNN
cuda
Train on 568006 samples, validate on 63112 samples, 2219 steps per epoch
Epoch 1/20
15s - loss:  0.7737 - mse:  0.7736 - val_mse:  0.4960
Epoch 2/20
12s - loss:  0.4756 - mse:  0.4756 - val_mse:  0.4877
Epoch 3/20
12s - loss:  0.4553 - mse:  0.4552 - val_mse:  0.4836
Epoch 4/20
13s - loss:  0.4413 - mse:  0.4413 - val_mse:  0.4853
Epoch 5/20
13s - loss:  0.4299 - mse:  0.4299 - val_mse:  0.4906
Epoch 6/20
12s - loss:  0.4202 - mse:  0.4202 - val_mse:  0.4974
Epoch 7/20
12s - loss:  0.4104 - mse:  0.4103 - val_mse:  0.5037
Epoch 8/20
12s - loss:  0.4006 - mse:  0.4005 - val_mse:  0.5103
Epoch 9/20
13s - loss:  0.3922 - mse:  0.3921 - val_mse:  0.5199
Epoch 10/20
12s - loss:  0.3853 - mse:  0.3851 - val_mse:  0.5207
Epoch 11/20
13s - loss:  0.3792 - mse:  0.3790 - val_mse:  0.5339
Epoch 12/20
12s - loss:  0.3741 - mse:  0.3739 - val_mse:  0.5404
Epoch 13/20
13s - loss:  0.3696 - mse:  0.3694 - val_mse:  0.5473
Epoch 14/20
12s - loss:  0.3653 - mse: 

##  3 Recent NN-based Methods
* **Attention Factorization Machines(AFM)**
* Collaborative Memory Networks (CMN)
* **xDeepFM**
* **Deep Interest Network (DIN)** *done for movielens*
* DeepGBM

In [15]:
#recent nn-based approach
from deepctr_torch.models import AFM
from deepctr_torch.models import xDeepFM
from deepctr_torch.models import DIN

# Din
import torch_rechub
from torch_rechub.utils.data import create_seq_features, df_to_dict, DataGenerator
from torch_rechub.basic.features import DenseFeature, SparseFeature, SequenceFeature
from torch_rechub.models.ranking import DIN
from torch_rechub.trainers import CTRTrainer


In [22]:

# Din
# need to have 'time' variable, so only movielens can be used
# https://www.cnblogs.com/junwei-kuang/p/DIN-DeepFM.html
# feature engeering
# 8 mins
# 7 epochs
data = Movielens['user_movie'].copy(deep = True)
data.columns = ['user_id', 'item_id', 'cate_id', 'time']


train, val, test = create_seq_features(data, seq_feature_col=['item_id', 'cate_id'], drop_short=3)
# get size of category data
n_users, n_items, n_cates = data["user_id"].max(), data["item_id"].max(), data["cate_id"].max()

features = [SparseFeature("target_item", vocab_size=n_items + 2, embed_dim=8), 
            SparseFeature("target_cate", vocab_size=n_cates + 2, embed_dim=8),
            SparseFeature("user_id", vocab_size=n_users + 2, embed_dim=8)]
target_features = features  
history_features = [
    SequenceFeature("history_item", vocab_size=n_items + 2, embed_dim=8, pooling="concat", shared_with="target_item"),
    SequenceFeature("history_cate", vocab_size=n_cates + 2, embed_dim=8, pooling="concat", shared_with="target_cate")]


train = df_to_dict(train)
val = df_to_dict(val)
test = df_to_dict(test)

train_y, val_y, test_y = train["label"], val["label"], test["label"]

del train["label"]
del val["label"]
del test["label"]
train_x, val_x, test_x = train, val, test

dg = DataGenerator(train_x, train_y)
train_dataloader, val_dataloader, test_dataloader = dg.generate_dataloader(x_val=val_x, y_val=val_y, x_test=test_x, y_test=test_y, batch_size=16)



# 定义模型，模型的参数需要我们之前的feature类，用于构建模型的输入层，mlp指定模型后续DNN的结构，attention_mlp指定attention层的结构
model = DIN(features=features, history_features=history_features, target_features=target_features, mlp_params={"dims": [256, 128]}, attention_mlp_params={"dims": [256, 128]})  # 这里的 features = target_features，理论上是不合理的，是为了特征太少来凑数？

ctr_trainer = CTRTrainer(model, optimizer_params={"lr": 1e-3, "weight_decay": 1e-3}, n_epoch=config['n_epoch'], earlystop_patience=4, device='cuda', model_path='./') 
ctr_trainer.fit(train_dataloader, val_dataloader)

auc = ctr_trainer.evaluate(ctr_trainer.model, test_dataloader)
pred_y = np.array(ctr_trainer.predict(model, test_dataloader))
r = sqrt(mean_squared_error(test_y, pred_y))

test_y = np.where(test_y > 3, 1, 0)
pred_y = np.where(pred_y > 3, 1, 0)

n = ndcg_score(test_y.reshape(1, -1), pred_y.reshape(1, -1), k=10)
re = recall_score(test_y.reshape(1, -1).squeeze(), pred_y.reshape(1, -1).squeeze())
name = 'movieDIN' 
print(f'test auc: {auc}')

with open("./metrics/rmse.txt", "r") as fp:
    rmse = json.load(fp)
with open("./metrics/ndcg.txt", "r") as fp:
    ndcg_10 = json.load(fp)
with open("./metrics/recall.txt", "r") as fp:
    recall_10 = json.load(fp)
    
rmse[name] = r
ndcg_10[name] = n
recall_10[name] = re
print(r, n, re, sep = '\n')

# with open("./metrics/rmse.txt", "w") as fp:
#     json.dump(rmse, fp)
# with open("./metrics/ndcg.txt", "w") as fp:
#     json.dump(ndcg_10, fp)
# with open("./metrics/recall.txt", "w") as fp:
#     json.dump(recall_10, fp)

epoch: 0


validation: 100%|██████████| 118/118 [00:01<00:00, 104.81it/s]


epoch: 0 validation: auc: 0.8643540785539259
epoch: 1


validation: 100%|██████████| 118/118 [00:01<00:00, 104.64it/s]


epoch: 1 validation: auc: 0.884261888402461
epoch: 2


...

validation: 100%|██████████| 118/118 [00:01<00:00, 101.93it/s]


epoch: 19 validation: auc: 0.9676884652105316


predict: 100%|██████████| 118/118 [00:01<00:00, 107.95it/s]

test auc: 0.9748248240931393
0.2500980843297598
0.0
0.0



/home/rita/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Results

In [17]:
with open("./metrics/rmse.txt", "r") as fp:
    rmse = json.load(fp)
with open("./metrics/ndcg.txt", "r") as fp:
    ndcg_10 = json.load(fp)
with open("./metrics/recall.txt", "r") as fp:
    recall_10 = json.load(fp)


In [20]:
iterables = [['book', 'movie', 'business'], ['rmse', 'recall_10', 'ndcg_10']]
# temp = pd.DataFrame(columns = ['book', 'movie', 'business'], index = model)
model = ['UCF-s', 'UCF-p', 'ICF-s', 'ICF-p', 'MF', 'FM', 'IPNN', 'OPNN', 'PNN', 'CCPM', 'WDL', 'DCN', 'NFM', 'DeepFM', 'AFM', 'DIN']
a = pd.MultiIndex.from_product(iterables, names = ['data', 'metrics'])
temp = pd.DataFrame(columns = a, index = model)
for i in temp.columns :
    for j in temp.index :
        for k in rmse.keys() :   
            if (i[0] in k) and (j in k) :
                t = sum([a.isupper() for a in k])
                if (t == 2) and (j == 'FM'): 
                    temp.at['FM', (i[0], 'rmse')] = round(rmse[k], 4)
                elif j != 'FM': 
                    # print(j)
                    temp.at[j, (i[0], 'rmse')] = round(rmse[k], 4)
                    
        for k in ndcg_10.keys() :   
            if (i[0] in k) and (j in k) :
                t = sum([a.isupper() for a in k])
                if (t == 2) and (j == 'FM'): 
                    temp.at['FM', (i[0], 'ndcg_10')] = round(ndcg_10[k], 4)
                elif j != 'FM': 
                    # print(j)
                    temp.at[j, (i[0], 'ndcg_10')] = round(ndcg_10[k], 4)
                    
        for k in recall_10.keys() :   
            if (i[0] in k) and (j in k) :
                t = sum([a.isupper() for a in k])
                if (t == 2) and (j == 'FM'): 
                    temp.at['FM', (i[0], 'recall_10')] = round(recall_10[k], 4)
                elif j != 'FM': 
                    # print(j)
                    temp.at[j, (i[0], 'recall_10')] = round(recall_10[k], 4)

# for i in temp.columns :
#     for j in temp.index :
#         for k in ndcg_10.keys() :   
#             if (i[0] in k) and (j in k) :
#                 t = sum([a.isupper() for a in k])
#                 if (t == 2) and (j == 'FM'): 
#                     temp.at['FM', (i[0], 'ndcg_10')] = round(ndcg_10[k], 4)
#                 elif j != 'FM': 
#                     # print(j)
#                     temp.at[j, (i[0], 'ndcg_10')] = round(ndcg_10[k], 4)

print(temp)
# temp.to_csv('results.csv')
temp.to_csv('results_round4.csv', float_format='%.4f')

data       book                     movie                   business   
metrics    rmse recall_10 ndcg_10    rmse recall_10 ndcg_10     rmse   
UCF-s    0.7565       NaN     NaN  1.0173       NaN     NaN   1.1039  \
UCF-p    0.7739       NaN     NaN   1.012       NaN     NaN   1.1652   
ICF-s    0.7563       NaN     NaN  1.0172       NaN     NaN   1.1049   
ICF-p    0.7736       NaN     NaN   1.011       NaN     NaN    1.165   
MF        0.486     0.994  0.7639    0.58    0.9269  0.6347    0.557   
FM       0.7063     0.999  0.7587  0.9516     0.942  0.6238   1.0324   
IPNN     0.7714    0.9808  0.7702  1.1151    0.8375   0.639   1.2151   
OPNN      0.758    0.9803  0.7716  1.0561    0.8767  0.6347   1.1732   
PNN      0.7598    0.9763  0.7732  1.1223    0.8485  0.6344   1.1627   
CCPM     0.7356     0.986  0.7721  1.0622    0.8872  0.6287   1.0883   
WDL      0.7139    0.9885  0.7715  1.0387     0.858  0.6488   1.0656   
DCN       0.732    0.9882    0.77  1.0405    0.8937  0.6296   1.

In [21]:
temp = pd.read_csv('results_round4.csv', header=[0,1], index_col = [0])
print(temp)

data       book                     movie                   business   
metrics    rmse recall_10 ndcg_10    rmse recall_10 ndcg_10     rmse   
UCF-s    0.7565       NaN     NaN  1.0173       NaN     NaN   1.1039  \
UCF-p    0.7739       NaN     NaN  1.0120       NaN     NaN   1.1652   
ICF-s    0.7563       NaN     NaN  1.0172       NaN     NaN   1.1049   
ICF-p    0.7736       NaN     NaN  1.0110       NaN     NaN   1.1650   
MF       0.4860    0.9940  0.7639  0.5800    0.9269  0.6347   0.5570   
FM       0.7063    0.9990  0.7587  0.9516    0.9420  0.6238   1.0324   
IPNN     0.7714    0.9808  0.7702  1.1151    0.8375  0.6390   1.2151   
OPNN     0.7580    0.9803  0.7716  1.0561    0.8767  0.6347   1.1732   
PNN      0.7598    0.9763  0.7732  1.1223    0.8485  0.6344   1.1627   
CCPM     0.7356    0.9860  0.7721  1.0622    0.8872  0.6287   1.0883   
WDL      0.7139    0.9885  0.7715  1.0387    0.8580  0.6488   1.0656   
DCN      0.7320    0.9882  0.7700  1.0405    0.8937  0.6296   1.